# Wines Points prediction 

## 

In [1]:
%load_ext autoreload
%autoreload 2
import sys; sys.path.append('../')

Here we will try to predict the points a wine will get based on known characteristics (i.e. features, in the ML terminology). The mine point in this stage is to establish a simple, ideally super cost effective, basline.
In the real world there is a tradeoff between complexity and perforamnce, and the DS job, among others, is to present a tradeoff tables of what performance is achivalbel at what complexity level. 

to which models with increased complexity and resource demands will be compared. Complexity should then be translated into cost. For example:
 * Compute cost 
 * Maintenance cost
 * Serving costs (i.e. is new platform needed?) 
 

## Loading the data

In [2]:
import pandas as pd
#import cufflinks as cf; cf.go_offline()

In [3]:
wine_reviews = pd.read_csv("C:\\Users\\eranb\Python_ML_2023\\data\\winemag-data-130k-v2.csv")
wine_reviews.shape


(129971, 14)

In [6]:
wine_reviews.duplicated().sum()

0

In [4]:
#wine_reviews1 = wine_reviews[['description', 'points']].copy()


In [4]:
wine_reviews91=wine_reviews.drop(['taster_twitter_handle','Unnamed: 0','title','region_1','region_2','country','designation','province'], axis=1)

In [5]:

wine_reviews91.dropna(inplace=True)

wine_reviews91.head()

,description,points,price,taster_name,variety,winery
1,"This is ripe and fruity, a wine that is smooth...",87,15.0,Roger Voss,Portuguese Red,Quinta dos Avidagos
2,"Tart and snappy, the flavors of lime flesh and...",87,14.0,Paul Gregutt,Pinot Gris,Rainstorm
3,"Pineapple rind, lemon pith and orange blossom ...",87,13.0,Alexander Peartree,Riesling,St. Julian
4,"Much like the regular bottling from 2012, this...",87,65.0,Paul Gregutt,Pinot Noir,Sweet Cheeks
5,Blackberry and raspberry aromas show a typical...,87,15.0,Michael Schachner,Tempranillo-Merlot,Tandem


In [7]:
wine_reviews91 = wine_reviews91.drop_duplicates()

wine_reviews91.shape

(88319, 6)

In [8]:
wine_reviews92=wine_reviews91.sample(1000)

In [37]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW

df = wine_reviews92

scaler = MinMaxScaler(feature_range=(0, 1))
df['points'] = scaler.fit_transform(df['points'].values.reshape(-1, 1))

train_texts, test_texts, train_labels, test_labels = train_test_split(df['description'], df['points'], test_size=0.2, random_state=42)

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(test_texts), truncation=True, padding=True)

train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_encodings['input_ids']),
                                               torch.tensor(train_encodings['attention_mask']),
                                               torch.tensor(train_labels.values))
test_dataset = torch.utils.data.TensorDataset(torch.tensor(test_encodings['input_ids']),
                                              torch.tensor(test_encodings['attention_mask']),
                                              torch.tensor(test_labels.values))

epochs = 3
batch_size = 16
learning_rate = 2e-5

model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        inputs = {'input_ids': batch[0].to(device),
                  'attention_mask': batch[1].to(device),
                  'labels': batch[2].to(device)}
        
        optimizer.zero_grad()
        outputs = model(**inputs)
        loss = outputs.loss
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
        scheduler.step()
    
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1} - Average training loss: {avg_train_loss:.4f}")



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.

ValueError: Target size (torch.Size([16])) must be the same as input size (torch.Size([16, 2]))

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW

df = wine_reviews92

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

MAX_LENGTH = 512  
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 8
EPOCHS = 3
LEARNING_RATE = 2e-5

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

train_encodings = tokenizer.batch_encode_plus(
    train_df["description"].tolist(),
    truncation=True,
    padding=True,
    max_length=MAX_LENGTH,
    return_tensors="pt"
)
test_encodings = tokenizer.batch_encode_plus(
    test_df["description"].tolist(),
    truncation=True,
    padding=True,
    max_length=MAX_LENGTH,
    return_tensors="pt"
)

train_dataset = torch.utils.data.TensorDataset(
    train_encodings["input_ids"],
    train_encodings["attention_mask"],
    torch.tensor(train_df["points"].tolist())
)
test_dataset = torch.utils.data.TensorDataset(
    test_encodings["input_ids"],
    test_encodings["attention_mask"],
    torch.tensor(test_df["points"].tolist())
)

# Load the pre-trained DistilBERT model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=100)

train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False
)


optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(EPOCHS):
    model.train()
    train_loss = 0

    for batch in train_loader:
        input_ids, attention_mask, labels = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_train_loss = train_loss / len(train_loader)

    




Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.

In [21]:
model.eval()
val_loss = 0
predictions = []
true_labels = []

for batch in test_loader:
    input_ids, attention_mask, labels = batch
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

    val_loss += loss.item()
    predictions.extend(torch.argmax(logits, dim=1).tolist())
    true_labels.extend(labels.tolist())

avg_val_loss = val_loss / len(test_loader)
accuracy = accuracy_score(true_labels, predictions)

print(f"Epoch {epoch+1}:")
print(f"  Train Loss: {avg_train_loss:.4f}")
print(f"  Validation Loss: {avg_val_loss:.4f}")
print(f"  Validation Accuracy: {accuracy:.4f}")


IndexError: Target 100 is out of bounds.

In [ ]:
#wine_reviews92

In [35]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

model_name = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name)

df = wine_reviews92
texts = df['description'].tolist()
labels = df['points'].tolist()

num_classes = max(labels) - min(labels) + 1

train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

train_encodings = tokenizer(train_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

train_dataset = torch.utils.data.TensorDataset(torch.tensor(train_encodings['input_ids']),
                                               torch.tensor(train_encodings['attention_mask']), train_labels)
test_dataset = torch.utils.data.TensorDataset(torch.tensor(test_encodings['input_ids']),
                                              torch.tensor(test_encodings['attention_mask']), test_labels)

batch_size = 16
epochs = 3
learning_rate = 1e-5

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

model.resize_token_embeddings(len(tokenizer))
model.classifier = torch.nn.Linear(model.config.hidden_size, num_classes)

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = torch.nn.CrossEntropyLoss()

model.train()
for epoch in range(epochs):
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

model.eval()
total_loss = 0
predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        batch_loss = loss_fn(logits.squeeze(), labels)
        total_loss += batch_loss.item() * input_ids.size(0)  # Multiply by batch size
        predictions.extend(torch.argmax(logits, dim=1).tolist())

average_loss = total_loss / len(test_dataset)
print("Average Loss:", average_loss)

predictions = [int(round(p)) for p in predictions]


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.

ValueError: Expected input batch_size (168) to match target batch_size (16).

In [ ]:
wine_reviews2s.shape

In [ ]:
import sys
print(sys.executable)

In [23]:
import torch

print(torch.__version__)

2.0.1+cpu


In [9]:
import transformers

In [ ]:
#wine_reviews1s

In [10]:
!pip install torch transformers

In [ ]:
data = wine_reviews2s

In [16]:
df =wine_reviews2s

In [24]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW


In [20]:
df =wine_reviews2s

In [21]:
X = df['description']
y = df['points']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

encoded_inputs_train = tokenizer(list(X_train), truncation=True, padding=True, max_length=256, return_tensors='pt')
encoded_inputs_test = tokenizer(list(X_test), truncation=True, padding=True, max_length=256, return_tensors='pt')

input_ids_train = encoded_inputs_train['input_ids']
attention_mask_train = encoded_inputs_train['attention_mask']
input_ids_test = encoded_inputs_test['input_ids']
attention_mask_test = encoded_inputs_test['attention_mask']

In [23]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=1)

optimizer = AdamW(model.parameters(), lr=1e-5)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

input_ids_train = input_ids_train.to(device)
attention_mask_train = attention_mask_train.to(device)
y_train = y_train.to(device)

model.to(device)
model.train()

for epoch in range(5):
    outputs = model(input_ids=input_ids_train, attention_mask=attention_mask_train, labels=y_train.unsqueeze(1))
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.weight', 'classifier.

AttributeError: 'Series' object has no attribute 'to'

In [44]:
wine_reviews2.description[131],wine_reviews2.points[131]

('Attractive mid-gold color. Compelling layers of tangy citrus and stone fruit aromas with notes of honey and earthy minerality. Intense flavors of honey, tangy orange citrus, sweet peach and apricot that are both rich and juicy.  Crisp acidity provides a perfect balance for the slight sweetness on the finish. Taut and focused.',
 91)